## Market Basket Analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
import missingno as mn
from scipy.stats import skew
from scipy.stats import kurtosis
import statsmodels.api as sm
import pylab as py
import import_ipynb
import Template as t
import Assignment_7_template as t2
import Assignment_7_template_Copy1 as t3

importing Jupyter notebook from Template.ipynb
importing Jupyter notebook from Assignment_7_template.ipynb
importing Jupyter notebook from Assignment_7_template_Copy1.ipynb


In [2]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
df=t.load_data('csv')

Enter path of fileproject_data.csv


In [4]:
df.head()

,customer_id,customer_store_name,customer_shipping_address,order_id,channel,order_placement_date,order_number,picked_timestamp,drop_timestamp,lat,...,locus_task_id,item_id,item_name,shipped_quantity,delivered_quantity,shipped_status,delivered_status,return_reason,unit_price,total_amount_expected
0,7eyDS7OyAs8DL5jRxiiHyb,Yousuf Store,MPR colony,171555b4-71fe-497e-a369-9f5a993cc89c,ADMIN_COMMANDO,2020-12-26 11:28:1608982086,20361239366,2020-12-30 06:24:1609309466,2020-12-30 13:09:1609333794,0.0,...,26fa7a81-b250-49c3-af2e-c7acff7f816d,1655,NaN,1,1,DELIVERED,DELIVERED,NaN,110.0,110.0
1,3964447752348,Manzoor Maqsood Wholesale,"Islamia Goth, Karachi",420a4e42-83ea-4401-a7ad-4a7bed1eef45,TELESALES_COMMANDO,2020-12-26 06:38:1608964711,20361966659,1970-01-01 00:00:1,1970-01-01 00:00:1,0.0,...,f6a2078b-d2cd-4748-bde6-6dfbffa3bd5d,1300,NaN,0,0,NOT_DELIVERED,NOT_DELIVERED,NaN,101600.0,0.0
2,4053006090396,nazar,Gulshen iqbal 14 askari park edhi wali gali,6b1832ee-1507-4f23-88e1-914b8f463eac,CUSTOMER_APP,2020-12-27 12:20:1609071638,20362569994,2021-01-01 06:14:1609481667,2021-01-01 08:40:1609490422,0.0,...,56c557e5-5837-4970-8582-61c0a27a8e88,1657,NaN,6,6,DELIVERED,DELIVERED,NaN,312.0,1872.0
3,4053006090396,nazar,Gulshen iqbal 14 askari park edhi wali gali,6b1832ee-1507-4f23-88e1-914b8f463eac,CUSTOMER_APP,2020-12-27 12:20:1609071638,20362569994,2021-01-01 06:14:1609481667,2021-01-01 08:40:1609490422,0.0,...,56c557e5-5837-4970-8582-61c0a27a8e88,1456,NaN,35,35,DELIVERED,DELIVERED,NaN,1855.0,64925.0
4,4053006090396,nazar,Gulshen iqbal 14 askari park edhi wali gali,6b1832ee-1507-4f23-88e1-914b8f463eac,CUSTOMER_APP,2020-12-27 12:20:1609071638,20362569994,2021-01-01 06:14:1609481667,2021-01-01 08:40:1609490422,0.0,...,56c557e5-5837-4970-8582-61c0a27a8e88,1463,NaN,2,2,DELIVERED,DELIVERED,NaN,7800.0,15600.0


In [5]:
df2=df[['order_id','item_name','shipped_quantity']]

In [6]:
df2

,order_id,item_name,shipped_quantity
0,171555b4-71fe-497e-a369-9f5a993cc89c,NaN,1
1,420a4e42-83ea-4401-a7ad-4a7bed1eef45,NaN,0
2,6b1832ee-1507-4f23-88e1-914b8f463eac,NaN,6
3,6b1832ee-1507-4f23-88e1-914b8f463eac,NaN,35
4,6b1832ee-1507-4f23-88e1-914b8f463eac,NaN,2
...,...,...,...
518883,2ce3d66a-8090-4386-bc4a-49380ef2ca9b,Hamdard Rooh Afza 800 ml,36
518884,2ce3d66a-8090-4386-bc4a-49380ef2ca9b,Nestle Everyday 375 gm Pack,3
518885,2ce3d66a-8090-4386-bc4a-49380ef2ca9b,Nestle Bunyad 260 gm Pack,5
518886,2ce3d66a-8090-4386-bc4a-49380ef2ca9b,Olper's Full Cream Milk 1.5 L Pack,40


In [7]:
df2=df2.loc[~df2['item_name'].isnull()]

In [8]:
df2=df2.loc[df2['shipped_quantity']!=0]

In [9]:
df2.reset_index(inplace=True)

In [10]:
df2.drop(columns=['index'],inplace=True)

In [11]:
df2

,order_id,item_name,shipped_quantity
0,fc7447ca-2123-454b-baf9-ad13b8fd9fb8,Mezan Canola Oil 1 Litre Carton,100
1,4b4cd91c-1e4f-435f-b1ed-2280f0e6e0db,Ahla Banaspati Ghee 900 gm Carton,10
2,1601f86f-5063-41f4-a749-ad555abf8f57,Dalda Banaspati Ghee 1 Kg Carton,50
3,1601f86f-5063-41f4-a749-ad555abf8f57,Dalda Cooking Oil 1 Litre Carton,100
4,1601f86f-5063-41f4-a749-ad555abf8f57,Tullo Cooking Oil 1 Litre Carton,25
...,...,...,...
454799,2ce3d66a-8090-4386-bc4a-49380ef2ca9b,Hamdard Rooh Afza 800 ml,36
454800,2ce3d66a-8090-4386-bc4a-49380ef2ca9b,Nestle Everyday 375 gm Pack,3
454801,2ce3d66a-8090-4386-bc4a-49380ef2ca9b,Nestle Bunyad 260 gm Pack,5
454802,2ce3d66a-8090-4386-bc4a-49380ef2ca9b,Olper's Full Cream Milk 1.5 L Pack,40


In [12]:
df2['item_name']=df2['item_name'].str.strip()

In [13]:
basket = (df2.groupby(['order_id', 'item_name'])['shipped_quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('order_id'))
basket.head(5)

item_name,1121 Ponia 25 Kg,1121 Ponia 25 Kg (2 Bags),1121 Ponia 25 Kg (3 Bags),1121 Ponia 25 Kg x 2 Bags,1121 Ponia 5 Kg,1121 Ponia 50 Kg,1121 Premium Steam Basmati 25 Kg,1121 Raw Basmati 25 Kg,1121 Raw Basmati 25 Kg (2 Bags),1121 Sella A 25 Kg,...,Yellow Peas 25 Kg (2 Bags),Zeera Indian 1 Kg,Zeera Indian 1 Kg x3,Zeera Kaala 1 Kg,Zeera Pakistani 1 Kg,Zeera Pakistani 1 Kg x5,Zeera Pisa 1 Kg,Zong - 100 x 6,Zong - 1000,Zong - 500
order_id,,,,,,,,,,,,,,,,,,,,,
000049fc-0890-421e-901d-806c6b3042fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0003e7a5-8e36-4264-9650-e5b7a38f029a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0003fe98-a7fb-4d70-a242-79c63bfafd0b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0005e71a-91c3-410e-8254-b8183637daf6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00087cbc-576e-49f6-be3a-73255c2d99fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets

item_name,1121 Ponia 25 Kg,1121 Ponia 25 Kg (2 Bags),1121 Ponia 25 Kg (3 Bags),1121 Ponia 25 Kg x 2 Bags,1121 Ponia 5 Kg,1121 Ponia 50 Kg,1121 Premium Steam Basmati 25 Kg,1121 Raw Basmati 25 Kg,1121 Raw Basmati 25 Kg (2 Bags),1121 Sella A 25 Kg,...,Yellow Peas 25 Kg (2 Bags),Zeera Indian 1 Kg,Zeera Indian 1 Kg x3,Zeera Kaala 1 Kg,Zeera Pakistani 1 Kg,Zeera Pakistani 1 Kg x5,Zeera Pisa 1 Kg,Zong - 100 x 6,Zong - 1000,Zong - 500
order_id,,,,,,,,,,,,,,,,,,,,,
000049fc-0890-421e-901d-806c6b3042fe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0003e7a5-8e36-4264-9650-e5b7a38f029a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0003fe98-a7fb-4d70-a242-79c63bfafd0b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0005e71a-91c3-410e-8254-b8183637daf6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00087cbc-576e-49f6-be3a-73255c2d99fe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff1aa3-a993-4a74-9ff4-247a8eb23e8a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ffff1d93-cf67-4681-8b11-5dcdafaff28c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ffff8db5-7e8f-4fb7-982a-3f7b1f12faa3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
frequent_itemsets = apriori(basket_sets, min_support=0.02, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.029192,(2.5 Chakee 10 Kg - Champion)
1,0.028556,(2.5 Chakee 50 Kg - Champion)
2,0.020798,(Ariel Washing Powder 0.5 Kg Pack)
3,0.058653,(Ariel Washing Powder 35 gm Sachet x 12)
4,0.033702,(Bona Papa Plus Large 50pc)
5,0.028104,(Bona Papa Plus Medium 50pc)
6,0.020012,(Brite Max Washing Powder 35 gm Sachet x 6)
7,0.152279,(Cheeni 50 Kg - Local)
8,0.022545,(Cup Shup Tea Whitener 225 ml Carton (27 pcs))
9,0.026142,(Hamdard Rooh Afza 800 ml)


In [16]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Surf Excel Washing Powder 35 gm Sachet x 6),(Ariel Washing Powder 35 gm Sachet x 12),0.054468,0.058653,0.027762,0.509695,8.690008,0.024567,1.919922
1,(Ariel Washing Powder 35 gm Sachet x 12),(Surf Excel Washing Powder 35 gm Sachet x 6),0.058653,0.054468,0.027762,0.473328,8.690008,0.024567,1.795295
2,(Knorr Noodles Chatt Patta 31.5 gm x 6),(Knorr Noodles Chicken 31.5 gm x 6),0.059963,0.048655,0.031661,0.528010,10.852074,0.028744,2.015602
3,(Knorr Noodles Chicken 31.5 gm x 6),(Knorr Noodles Chatt Patta 31.5 gm x 6),0.048655,0.059963,0.031661,0.650726,10.852074,0.028744,2.691404
4,(Nestle Bunyad 26 gm Sachet x 12),(Nestle Everyday 15 gm Sachet x 12),0.047067,0.077378,0.020258,0.430403,5.562335,0.016616,1.619781
5,(Nestle Everyday 15 gm Sachet x 12),(Nestle Bunyad 26 gm Sachet x 12),0.077378,0.047067,0.020258,0.261802,5.562335,0.016616,1.290891
